<a href="https://colab.research.google.com/github/trefftzc/partition_COLAB_notebooks/blob/main/Partition_with_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a small instance of the partition problem with PySpark

The multiset will be hardcoded.
The size of the problem is 10.
2^10 is 1024

In [ ]:
!ls


sample_data


Create a SparkContext to be able to execute.

In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[2]")

sc = SparkContext.getOrCreate(spark_conf)

In [ ]:
sc

<SparkContext master=local[2] appName=YourTest>

Start by creating a list that contains the numbers between 1 and 1024

In [ ]:
nums = list(range(1, 1024))
len(nums)

1023

Now create an RDD (Resilient Distributed Dataset)

In [ ]:
nums_rdd = sc.parallelize(nums)

In [ ]:
nums_rdd.take(5)

[1, 2, 3, 4, 5]

Define the function that determines if a number encodes a partition that is a solution to the problem or not.
The array for the instance of the problem is hardcoded here.

In [ ]:
def evaluatePartition(  value ) :
   array = [1,1,1,1,1,1,1,1,1,9]
   n = 10
   sum0s = 0
   sum1s = 0
   mask = 1
   for i in range(0,n):
     if ((mask & value) != 0):
       sum1s = sum1s + array[i]
     else:
       sum0s = sum0s + array[i]
     mask = mask * 2

   if (sum0s == sum1s):
     return value
   else:
     return 0

In [ ]:
print(evaluatePartition(1))

0


Now apply the function evaluatePartition to the nums_rdd and store the results in a different rdd:

In [ ]:
result_rdd = nums_rdd.map(evaluatePartition)

Finally, find if any entry turned to be different from 0, a solution

In [ ]:
from operator import add
sol = result_rdd.reduce(max)
if sol == 0:
  print("This instance of the partition problem does not have a solution")
else:
  print("This instance of the partition problem does have a solution.")
print(sol)

This instance of the partition problem does have a solution.
512
